
## Minimax with Alpha-Beta Pruning
As seen in Lab 5, Minimax is an algorithm that makes optimal moves by minimizing the loss, assuming the other player is playing optimally. However, optimiality comes with the cost of exploring many game states, which can be costly in large search spaces. In this section you will implement Minimax with Alpah-Beta pruning for tic tac toe.

⚠️ **Academic Integrity Reminder**

AI-assisted code generation is not allowed for this assignment. Please disable AI code suggestions (e.g., Colab's AI or extensions like Copilot) while completing this assignment.

To disable AI suggestions in Colab:

- Go to `Tools > Settings > Editor`
- Uncheck `Show AI-powered inline completions`




### Introduction
Minimax will always find the solution, at the expense of exploring every game state. By using Alpha-Beta pruning, this search space is reduced. In this lab, you will implement this method for the game of tic tac toe.



Observe the following tic tac toe board:

In [ ]:
# Lab by Morgan Swanson, adapted by Rodrigo  Canaan
# Note: boards have to be np arrays. Passing a simple list whenre code expects a board will lead to errors.
import numpy as np
example_board = np.array([[' ', ' ', ' '],
                          [' ', ' ', ' '],
                          [' ', ' ', ' ']])
victory_x = np.array([['X', 'O', 'O'],
                      ['X', 'X', ' '],
                      ['X', ' ', 'O']])
tie = np.array([['X', 'X', 'O'],
               ['O', 'X', 'X'],
               ['X', 'O', 'O']])
print(example_board)
print(victory_x)
print(tie)

[[' ' ' ' ' ']
 [' ' ' ' ' ']
 [' ' ' ' ' ']]
[['X' 'O' 'O']
 ['X' 'X' ' ']
 ['X' ' ' 'O']]
[['X' 'X' 'O']
 ['O' 'X' 'X']
 ['X' 'O' 'O']]


***
### Get Score
This function calculates the game score to be either:
1. Max victory - value greater than zero
2. Min victory -value less than zero
3. Draw - value of zero
4. Incomplete - value of None

It scales by the depth of the search in order to delay losing as long as possible and win as quickly as possible.


In [ ]:
# Gets the final score of a board
# 1: game ended with a win for X
# -1 game ended with a win for O
# 0: game ended with a tie
# None: state is not terminal (game hasn't ended yet)
def get_score(board, depth=0):
    if (np.any(np.all(board == 'X', axis=0)) or
        np.any(np.all(board == 'X', axis=1)) or
        np.all(board.diagonal() == 'X') or
        np.all(np.fliplr(board).diagonal() == 'X')):
        # Max Victory
        return 1 * (1 / (1 + depth))
    elif (np.any(np.all(board == 'O', axis=0)) or
          np.any(np.all(board == 'O', axis=1)) or
          np.all(board.diagonal() == 'O') or
          np.all(np.fliplr(board).diagonal() == 'O')):
        # Min Victory
        return -1 * (1 / (1 + depth))
    elif not (board == ' ').any():
        # Draw
        return 0
    else:
        # Unfinished Game
        return None


tests = np.array([[['O', 'X', ' '],
                   ['O', 'O', 'X'],
                   ['X', ' ', 'O']],
                  [['X', 'O', 'O'],
                   ['O', 'X', 'O'],
                   [' ', ' ', 'X']],
                  [['O', 'O', 'X'],
                   ['X', 'X', 'O'],
                   ['O', 'X', 'O']],
                  [['O', 'X', ' '],
                   ['O', 'O', 'X'],
                   ['X', ' ', ' ']]])

for test in tests:
  print(test, get_score(test))

[['O' 'X' ' ']
 ['O' 'O' 'X']
 ['X' ' ' 'O']] -1.0
[['X' 'O' 'O']
 ['O' 'X' 'O']
 [' ' ' ' 'X']] 1.0
[['O' 'O' 'X']
 ['X' 'X' 'O']
 ['O' 'X' 'O']] 0
[['O' 'X' ' ']
 ['O' 'O' 'X']
 ['X' ' ' ' ']] None


***
### Get Possible Boards
This function generates all possible moves for the given board and player. It will place an 'X' for the maximizing player and 'O' for the minimizing player. Observe the generated boards.



In [ ]:
# Calculates all successor states to a given state. Returns the empty list if state is terminal.
def get_possible_moves(board, player):
    moves = []
    if get_score(board) is None:
      for (x, y), element in np.ndenumerate(board):
          if element == ' ':
              new_board = np.array(board, copy=True)
              new_board[x][y] = 'X' if player == 'max' else 'O'
              moves.append(new_board)
    return moves

example_moves = get_possible_moves(example_board, 'min') # test with victory_x or tie to see that it correctly handles terminal states
if not example_moves:
  print("No legal moves. State is terminal.")

else:
  for b in example_moves:
    print(b)

[['O' ' ' ' ']
 [' ' ' ' ' ']
 [' ' ' ' ' ']]
[[' ' 'O' ' ']
 [' ' ' ' ' ']
 [' ' ' ' ' ']]
[[' ' ' ' 'O']
 [' ' ' ' ' ']
 [' ' ' ' ' ']]
[[' ' ' ' ' ']
 ['O' ' ' ' ']
 [' ' ' ' ' ']]
[[' ' ' ' ' ']
 [' ' 'O' ' ']
 [' ' ' ' ' ']]
[[' ' ' ' ' ']
 [' ' ' ' 'O']
 [' ' ' ' ' ']]
[[' ' ' ' ' ']
 [' ' ' ' ' ']
 ['O' ' ' ' ']]
[[' ' ' ' ' ']
 [' ' ' ' ' ']
 [' ' 'O' ' ']]
[[' ' ' ' ' ']
 [' ' ' ' ' ']
 [' ' ' ' 'O']]


***
### Find Value
Your task is the implement value function, which will return the value of the board calcuated using minimax. If a node is not terminal, it will call this function recursively on its children. To recieve full credit, this function must use alpha beta pruning.  A skeleton has been provided for you:


* **board:** the tic tac toe board that must be evalutated
* **alpha:** the current alpha parameter of the search
* **beta:** the current beta parameter of the search
* **player:** the current player who is acting
* **depth:** how deep in the tree we are from the initial call. Affects scoring function (shallow winning nodes are more valuable than deep ones)




*Hint: Implement minimax with no pruning first, then add pruning once you're confident in the minimax implementation*


In [ ]:
# Finds the integer expected score of a game state using minimax wiht alpha-beta
def find_value(board, player, alpha=float('-inf'), beta=float('inf'), depth=0):
    return 0

***
### Find Best Move
We will now use our value finder in order to calculate the value of all possible moves and choose the best one.


In [ ]:
# Finds the best move (state with highest/lowest value)
from tqdm import tqdm_notebook as tqdm
def find_best_move(board, player):
    print("Deciding best move...")
    boards = get_possible_moves(board, player)
    print(boards)
    values = [find_value(board,
                         ('max' if player == 'min'
                          else 'min'))
              for board in tqdm(boards)]
    print(values)
    if None in values:
        raise ValueError('find_value should always return an integer.')
    if player == "max":
        policy_vector = np.array([1 if value == np.amax(values) else 0
                                  for value in values])
    else:
        policy_vector = np.array([1 if value == np.amin(values) else 0
                                  for value in values])
    # Normalize Probabilities
    policy_vector = policy_vector / np.count_nonzero(policy_vector)
    return boards[np.random.choice(np.arange(len(values)),
                                   1,
                                   p=policy_vector)[0]]

test_boards = [np.array([['X', 'O', ' '],
                ['O', 'X', ' '],
                [' ', ' ', ' ']])]
print(find_best_move(test_boards[0], 'max'))


Deciding best move...
[array([['X', 'O', 'X'],
       ['O', 'X', ' '],
       [' ', ' ', ' ']], dtype='<U1'), array([['X', 'O', ' '],
       ['O', 'X', 'X'],
       [' ', ' ', ' ']], dtype='<U1'), array([['X', 'O', ' '],
       ['O', 'X', ' '],
       ['X', ' ', ' ']], dtype='<U1'), array([['X', 'O', ' '],
       ['O', 'X', ' '],
       [' ', 'X', ' ']], dtype='<U1'), array([['X', 'O', ' '],
       ['O', 'X', ' '],
       [' ', ' ', 'X']], dtype='<U1')]


<ipython-input-53-8d5bd64189ac>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for board in tqdm(boards)]


  0%|          | 0/5 [00:00<?, ?it/s]

[0, 0, 0, 0, 0]
[['X' 'O' ' ']
 ['O' 'X' 'X']
 [' ' ' ' ' ']]


***
### Testing
In order to test our AI, we can play a few games against it:

In [ ]:
# Plays a match between  a human and the bot, asking the human for their move  via console
# To select the  role of the  bot (max or min), edit  line 11

def run_demo():
  board = np.array([[' ', ' ', ' '],
                    [' ', ' ', ' '],
                    [' ', ' ', ' ']])
  score = get_score(board)
  starting_player = "max" # Starting player. Change this to "min" if you want to test a position where the next player is O
  computer_players = ["max","min"] # List of computer-controlled players. Use []  for human vs human play, one of ["min"] or ["max"] for human vs bot, or ["min","max"] for bot vs bot.
  player = starting_player
  while score is None:
      if player in computer_players:
          board = find_best_move(board, player)
      else:
          move_entered = False
          while not move_entered:
              try:
                  print("Current board:")
                  print(board)
                  move = int(input('Choose a move for player {}...'.format(player))) - 1
                  if not 0 <= move <= 8:
                      print("Enter an integer between 1 and 9.\n")
                      continue
                  elif not board[move//3][move%3] == ' ':
                      print("That spot is already taken.\n")
                      continue
                  else:
                      board[move//3][move%3]= 'O' if player == "min" else "X"
                      move_entered = True
              except ValueError:
                  print("Enter an integer.\n")
      score = get_score(board)
      player = "min" if player == "max" else "max"
      print(board)
  if (score == 0):
      print("Draw")
  elif (score > 0):
      print("X wins")
  else:
      print("O wins")


run_demo()

Deciding best move...
[array([['X', ' ', ' '],
       [' ', ' ', ' '],
       [' ', ' ', ' ']], dtype='<U1'), array([[' ', 'X', ' '],
       [' ', ' ', ' '],
       [' ', ' ', ' ']], dtype='<U1'), array([[' ', ' ', 'X'],
       [' ', ' ', ' '],
       [' ', ' ', ' ']], dtype='<U1'), array([[' ', ' ', ' '],
       ['X', ' ', ' '],
       [' ', ' ', ' ']], dtype='<U1'), array([[' ', ' ', ' '],
       [' ', 'X', ' '],
       [' ', ' ', ' ']], dtype='<U1'), array([[' ', ' ', ' '],
       [' ', ' ', 'X'],
       [' ', ' ', ' ']], dtype='<U1'), array([[' ', ' ', ' '],
       [' ', ' ', ' '],
       ['X', ' ', ' ']], dtype='<U1'), array([[' ', ' ', ' '],
       [' ', ' ', ' '],
       [' ', 'X', ' ']], dtype='<U1'), array([[' ', ' ', ' '],
       [' ', ' ', ' '],
       [' ', ' ', 'X']], dtype='<U1')]


<ipython-input-53-8d5bd64189ac>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for board in tqdm(boards)]


  0%|          | 0/9 [00:00<?, ?it/s]

[0, 0, 0, 0, 0, 0, 0, 0, 0]
[[' ' ' ' ' ']
 [' ' 'X' ' ']
 [' ' ' ' ' ']]
Deciding best move...
[array([['O', ' ', ' '],
       [' ', 'X', ' '],
       [' ', ' ', ' ']], dtype='<U1'), array([[' ', 'O', ' '],
       [' ', 'X', ' '],
       [' ', ' ', ' ']], dtype='<U1'), array([[' ', ' ', 'O'],
       [' ', 'X', ' '],
       [' ', ' ', ' ']], dtype='<U1'), array([[' ', ' ', ' '],
       ['O', 'X', ' '],
       [' ', ' ', ' ']], dtype='<U1'), array([[' ', ' ', ' '],
       [' ', 'X', 'O'],
       [' ', ' ', ' ']], dtype='<U1'), array([[' ', ' ', ' '],
       [' ', 'X', ' '],
       ['O', ' ', ' ']], dtype='<U1'), array([[' ', ' ', ' '],
       [' ', 'X', ' '],
       [' ', 'O', ' ']], dtype='<U1'), array([[' ', ' ', ' '],
       [' ', 'X', ' '],
       [' ', ' ', 'O']], dtype='<U1')]


  0%|          | 0/8 [00:00<?, ?it/s]

...

[0, 0, 0, 0, 0, 0]
[[' ' 'O' 'O']
 [' ' 'X' 'X']
 [' ' ' ' ' ']]
Deciding best move...
[array([['X', 'O', 'O'],
       [' ', 'X', 'X'],
       [' ', ' ', ' ']], dtype='<U1'), array([[' ', 'O', 'O'],
       ['X', 'X', 'X'],
       [' ', ' ', ' ']], dtype='<U1'), array([[' ', 'O', 'O'],
       [' ', 'X', 'X'],
       ['X', ' ', ' ']], dtype='<U1'), array([[' ', 'O', 'O'],
       [' ', 'X', 'X'],
       [' ', 'X', ' ']], dtype='<U1'), array([[' ', 'O', 'O'],
       [' ', 'X', 'X'],
       [' ', ' ', 'X']], dtype='<U1')]


  0%|          | 0/5 [00:00<?, ?it/s]

[0, 0, 0, 0, 0]
[[' ' 'O' 'O']
 [' ' 'X' 'X']
 ['X' ' ' ' ']]
Deciding best move...
[array([['O', 'O', 'O'],
       [' ', 'X', 'X'],
       ['X', ' ', ' ']], dtype='<U1'), array([[' ', 'O', 'O'],
       ['O', 'X', 'X'],
       ['X', ' ', ' ']], dtype='<U1'), array([[' ', 'O', 'O'],
       [' ', 'X', 'X'],
       ['X', 'O', ' ']], dtype='<U1'), array([[' ', 'O', 'O'],
       [' ', 'X', 'X'],
       ['X', ' ', 'O']], dtype='<U1')]


  0%|          | 0/4 [00:00<?, ?it/s]

[0, 0, 0, 0]
[['O' 'O' 'O']
 [' ' 'X' 'X']
 ['X' ' ' ' ']]
O wins
